Firstly we will import all the dependencies 

In [15]:
import json
import os
import shutil
import cv2
import numpy as np
import os
# from matplotlib import pyplot as plt
# import time
# import mediapipe as mp
# import re

In [17]:
MAX_FRAME=164
PER_FRAME_FEATURE =1629
testUser = {'U4','U8'}
relativePath='RELATIVE'

In [22]:
def quantizeFrame(npy1629):
    
    # decide quantization levels allowable for the point
    # get the quantization string for the point
    #concatanate all the quantizatioin strings to get the total string for the frame
    
    npy_mat =npy1629.reshape(-1,3)
    totalString=''
 

    point=542
    for i in range (0,543):
        x=npy_mat[point][0]
        y=npy_mat[point][1]
        d=npy_mat[point][2]
        
        
        if point in range (522,543): #right hand points
            x_level =10
            y_level=10
            d_level=5
            
            
        elif point in range (501,522): #left hand
            x_level =10
            y_level=10
            d_level=5
        elif point in range (33,501): #face points
            #special emphasis to be put for mount points..... will do later
            x_level =5
            y_level=5
            d_level=3

        elif point in range (0,33):
            #point value comes from 32 to 0 as loop designed
            if point ==0: # nose   
                x_level =5
                y_level=5
                d_level=3
            
            elif point in range (1,11): # face points in pose
                x_level =5
                y_level=5
                d_level=3
            elif point in range (11,13):  # two shoulder points 
                x_level =5
                y_level=5
                d_level=3

           
            elif point ==13: #lefthand elbow 
                x_level =10
                y_level=10
                d_level=5

            elif point ==15: #lefthand wrist
                x_level =10
                y_level=10
                d_level=5
            elif point ==14: #righht elbow 
                x_level =10
                y_level=10
                d_level=5

            elif point ==16: #righht wrist
                x_level =10
                y_level=10
                d_level=5
                
            elif point in range (17,23) and point %2 ==1: #lefthand  finger tips
                x_level =10
                y_level=10
                d_level=5
            elif point in range (17,23) and point %2 ==0: #righthand  finger tips
                x_level =10
                y_level=10
                d_level=5
            elif point in range (23,25):  # two hips do nothing
                x_level =1
                y_level=1
                d_level=1
            elif point ==25: #left kneeee 
                x_level =1
                y_level=1
                d_level=1

            elif point ==27: #left ankle
                x_level =1
                y_level=1
                d_level=1
            elif point ==29: #left heel
                x_level =1
                y_level=1
                d_level=1

            elif point ==31: #left index
                x_level =1
                y_level=1
                d_level=1


            elif point ==26: #rightt kneeee 
                x_level =1
                y_level=1
                d_level=1

            elif point ==28: #righhtt ankle
                x_level =1
                y_level=1
                d_level=1

            elif point ==30: #righhtt heel
                x_level =1
                y_level=1
                d_level=1

            elif point ==32: #righhtt index
                x_level =1
                y_level=1
                d_level=1
            
        x =int(x*x_level)
        y =int(y*y_level)
        d =int(d*d_level)
        
        npy_mat[point][0]=x
        npy_mat[point][1]=y
        npy_mat[point][2]=d
        
        point=point-1




    return npy1629

In [23]:
def quantizeNpy (npy):
    # print('Q')
    myQLines=''
    npy164=npy.reshape(-1,PER_FRAME_FEATURE)
    # print(npy164.shape)
    for i in range(0,MAX_FRAME):
        nyp1629 =quantizeFrame(npy164[i])
    

In [24]:
crossValidationDataPaths=['../RELATIVE_NORMALIZED/NumericalData/RELATIVE/ML/Training/RightHand','../RELATIVE_NORMALIZED/NumericalData/RELATIVE/ML/Training/LeftHand']
testDataPaths=['../RELATIVE_NORMALIZED/NumericalData/RELATIVE/ML/Test/RightHand','../RELATIVE_NORMALIZED/NumericalData/RELATIVE/ML/Test/LeftHand']

relativeQ='../RELATIVE_QUANT_D5MAX/'

In [25]:




ml_instances_paths=[]
for path in crossValidationDataPaths:
    #print(path)
    classFolders= os.listdir(path)
    for classFolder in classFolders:
        classFolder_path=f'{path}/{classFolder}'
        classTrials =os.listdir(classFolder_path)
        for trial in classTrials:
            trialPath =f'{classFolder_path}/{trial}'
            #print(trialPath)
            ml_instances_paths.append(trialPath)



test_paths=[]
for path in testDataPaths:
    #print(path)
    classFolders= os.listdir(path)
    for classFolder in classFolders:
        classFolder_path=f'{path}/{classFolder}'
        classTrials =os.listdir(classFolder_path)
        for trial in classTrials:
            trialPath =f'{classFolder_path}/{trial}'
            #print(trialPath)
            test_paths.append(trialPath)





In [26]:

def getDirAndFN(path):
    dir_path=relativeQ
    label = path.split('/')
    i=0
    for i in range(5,len(label)-1):
        dir_path=dir_path+label[i]+'/'
    #print(dir_path)
    
    fileName =label[len(label)-1]
    return dir_path, fileName


In [27]:
def writeNpy(dir_path,file_name,npy):
    if os.path.exists(dir_path):
        pass
    else:
        os.makedirs(dir_path)
    np.save(dir_path+file_name,npy)
    

In [28]:
for ml_instances_path in ml_instances_paths:
    dir_path, file_name=getDirAndFN(ml_instances_path)
    
  
    
    npy =np.load(ml_instances_path)
    quantizeNpy(npy)
    writeNpy(dir_path,file_name,npy)
    # print(qlines)
   
    




for ml_instances_path in test_paths:
    dir_path, file_name=getDirAndFN(ml_instances_path)
    
   
    npy =np.load(ml_instances_path)
    quantizeNpy(npy)
    writeNpy(dir_path,file_name,npy)